In [1]:
import sys

sys.path.insert(0, '/mnt/data/molchanov/dltranz')

import torch
import torch.nn.functional as F
from torch.autograd import Function, Variable
import numpy as np
import random

import matplotlib.pyplot as plt

from domyshnik.models import *
from domyshnik.data import *
from domyshnik.constants import *
from domyshnik.utils import *
from domyshnik.losses import *

def draw(imgs):
        if isinstance(imgs, list):
            imgs = torch.stack(imgs)
        fig = plt.figure()
        rows, columns = 1, imgs.shape[0]
        for i in range(imgs.shape[0]):
            fig.add_subplot(rows, columns, i+1)
            if imgs[i].size(0) == 3:
                plt.imshow(imgs[i].transpose(0, 1).transpose(1, 2))
            else:
                plt.imshow(imgs[i])
        plt.show()
        
def draw_rgb(imgs):
    if isinstance(imgs, list):
        imgs = torch.stack(imgs)
    fig = plt.figure()
    rows, columns = 1, imgs.shape[0]
    for i in range(imgs.shape[0]):
        fig.add_subplot(rows, columns, i+1)
        plt.imshow(imgs[i].transpose(0, 1).transpose(1, 2))
    plt.show()
        
%matplotlib inline


model _params:
        CURRENT_PARAMS cifar10_metric_learning_global_basis
        N_AUGMENTS 5
        LEARNING_RATE 0.002
        GAMMA 0.9025
        BATCH_SIZE 128
        EPOCHS 50
        SAMPLING_STRATEGY HardNegativePair
        NEGATIVES_COUNT 5
        MARGING 0.5
        STEP_SIZE 5
        MODEL_POSTFIX cifar10_global_c150_caug5_iaug5_cmrg05_imrg05_basis
        ADD_INFO {'centroids_count': 150, 'losses': [{'name': 'BasisClusterisationLoss'}, {'name': 'ContrastiveLossOriginal_centroids', 'marging': 0.5, 'neg_count': 5, 'sampling_strategy': 'HardNegativePair'}, {'name': 'ContrastiveLossOriginal_images', 'marging': 0.5, 'neg_count': 5, 'sampling_strategy': 'HardNegativePairKlDiv'}], 'n_augs_imgs': 5}
        ERROR_RATE 0.5
       


In [3]:
t=torch.randint(0, 5, (4, 2, 5))
t

tensor([[[2, 2, 2, 3, 1],
         [4, 3, 2, 3, 3]],

        [[0, 1, 3, 0, 2],
         [2, 2, 4, 3, 3]],

        [[4, 1, 4, 0, 4],
         [4, 4, 3, 2, 4]],

        [[4, 2, 4, 4, 3],
         [4, 4, 3, 0, 3]]])

In [8]:
t.index_select(1, torch.Tensor([0]).long()).squeeze()

tensor([[2, 2, 2, 3, 1],
        [0, 1, 3, 0, 2],
        [4, 1, 4, 0, 4],
        [4, 2, 4, 4, 3]])

In [14]:
t1 = np.random.randn(10, 10)
t2 = np.random.randn(10, 10)
np.concatenate((t1, t2), 0).shape

(20, 10)

In [4]:
x = torch.Tensor([0.1, 0.2, 0.3, 0.5])
h = F.softmax(x, dim=0) * F.log_softmax(x, dim=0)
-h.sum()

tensor(1.3750)

In [5]:
(x * torch.log(x)).sum()

tensor(-1.2599)

In [7]:
d = torch.randn(10) + 1
F.kl_div(F.log_softmax(d), F.softmax(d), reduction='none')

/mnt/data/molchanov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  
/mnt/data/molchanov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/mnt/data/molchanov/miniconda3/lib/python3.7/site-packages/torch/nn/functional.py:1958: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


tensor(-2.8129e-09)

In [11]:
t1 = torch.zeros(150)
t1[0] = 1.0
t2 = torch.zeros(150)
t2[1] = 1.0


In [12]:
F.kl_div(F.log_softmax(t1), F.softmax(t2), reduction='none').sum()

/mnt/data/molchanov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.
/mnt/data/molchanov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


tensor(0.0113)

In [ ]:
KLpq = 